# Analyse von *Tiktok*-Daten, die mit `zeeschuimer` gesammelt wurden

In diesem Notebook schauen wir uns exemplarisch an, wie man automatisierte Textanalysen mit *TikTok*-Daten durchführen kann, die mit dem Tool [`zeeschuimer`](https://github.com/digitalmethodsinitiative/zeeschuimer) gesammelt wurden.

*Wichtiger Hinweis*: Ausgangspunkt sind `.csv`-Dateien, die entweder mit [4CAT](https://4cat.nl/) oder dem im [Repository zur Sammlung und Exploration von *TikTok*-Daten](https://github.com/jobreu/traktok-demo) inkludierten Parser für die mit `zeeschuimer` erstellten `.njdson`-Dateien generiert wurden.

Eine Alternative zu `zeeschuimer` für die Sammlung von *TikTok*-Daten ist das `R`-Paket [`traktok`](https://jbgruber.github.io/traktok/). Beispiele zu dessen Nutzung finden sich auch im o.g. [Notebook zur Sammlung und Exploration von *TikTok*-Daten](https://github.com/jobreu/traktok-demo).

## Datenimport

Zur Erinnerung: Die verwendete `.csv`-Datei muss über den Upload-Button (nach oben zeigender Pfeil) im File Explorer auf der linken Seite der Jupyter-Lab-Analyseumgebung hochgeladen werden.

In [ ]:
library(readr)
library(dplyr)

### Posts/Videos

In [ ]:
tiktok_posts <- read_csv("INSERT_FILE_NAME_HERE") # Namen der entsprechenden Datei (inkl. Dateiendung) einfügen

In [ ]:
names(tiktok_posts)

In [ ]:
glimpse(tiktok_posts)

Welche Accounts sind am häufigsten im Datensatz vertreten?

In [ ]:
tiktok_posts %>% 
  count(author_full) %>% 
  arrange(desc(n)) %>% 
  head(10)

### Kommentare (zu einzelnen Videos)

In [ ]:
tiktok_comments <- read_csv("INSERT_FILE_NAME_HERE") # Namen der entsprechenden Datei (inkl. Dateiendung) einfügen

In [ ]:
names(tiktok_comments)

In [ ]:
glimpse(tiktok_comments)

## Textanalyse: Posts

Nachfolgend bereiten wir die Daten zu den Posts/Videos für die automatisierte Textanalyse auf und führen dann erste deskriptive Analysen durch.

In [ ]:
library(quanteda)

### Erstellung eines Corpus

In [ ]:
tiktok_videos_corpus <- tiktok_posts %>% 
  select(id, author, author_full,
         timestamp, unix_timestamp,
         likes, comments, shares, plays,
         tiktok_url,
         body) %>% 
  corpus(docid_field = "id",
         text_field = "body")

### Tokenisierung & Entfernung von Stop Words

In [ ]:
tokens_tiktok_videos <- tokens(tiktok_videos_corpus,
                         remove_punct = TRUE,
                         remove_symbols = TRUE,
                         remove_numbers = TRUE,
                         remove_url = TRUE)

In [ ]:
tokens_tiktok_videos <- tokens_remove(tokens_tiktok_videos,
                                stopwords("de"))

In [ ]:
tokens_tiktok_videos

### Document-Feature-Matrix (DFM) erstellen

In [ ]:
dfm_tiktok_videos <- dfm(tokens_tiktok_videos)

In [ ]:
dfm_tiktok_videos

### Textdaten explorieren

In [ ]:
library(quanteda.textstats)

#### Worthäufigkeiten

In [ ]:
dfm_tiktok_videos %>%
  dfm_remove(pattern = c("dass",
                         "@*", "#*")) %>% # ohne User Tags und Hashtags
  textstat_frequency(n = 20)

#### Top Hashtags

In [ ]:
dfm_tag <- dfm_select(dfm_tiktok_videos, pattern = "#*")
toptag <- names(topfeatures(dfm_tag, 50))
head(toptag, 10)

#### Top User Tags

In [ ]:
dfm_users <- dfm_select(dfm_tiktok_videos, pattern = "@*")
topuser <- names(topfeatures(dfm_users, 50))
head(topuser, 10)

## Visualisierung

In [ ]:
library(quanteda.textplots)

#### Wortwolke

In [ ]:
dfm_tiktok_videos %>% 
  dfm_remove(pattern = c("dass",
                         "@*", "#*")) %>% 
  dfm_trim(min_termfreq = 50) %>%
  textplot_wordcloud()

#### Plot zu Worthäufigkeiten

In [ ]:
library(ggplot2)

In [ ]:
tstat_freq <- dfm_tiktok_videos %>% 
  dfm_remove(pattern = c("dass",
                         "@*", "#*")) %>%  
  textstat_frequency(n = 20)

ggplot(tstat_freq, aes(x = frequency, y = reorder(feature, frequency))) +
  geom_col() + 
  labs(x = "Frequency", y = "Feature") +
  scale_x_continuous(expand = expansion(mult = c(0, 0.05)))

#### Relative Worthäufigkeiten / Keyness

In [ ]:
tstat_key <- dfm_tiktok_videos %>% 
  dfm_remove(pattern = c("dass",
                         "@*", "#*")) %>%  
  textstat_keyness(target = dfm_tiktok_videos$author_full == "INSERT_ACCOUNT_NAME") # hier Namen des Accounts einfügen, der mit dem Rest des Corpus verglichen werden soll

textplot_keyness(tstat_key)

## Textanalyse: Kommentare

Analog zu den Aufbereitungs- und Analyseschritten für die Posts können wir auch für die Kommentare verfahren. Hier machen allerdings Analysen der Top Hashtags sowie eine Keyness-Analyse aufgrund der Struktur der Daten weniger Sinn, weshalb wir sie in den nachfolgenden Beispielen auch weglassen.

### Corpus

In [ ]:
tiktok_comments_corpus <- tiktok_comments %>% 
  select(id, thread_id,
         author, author_full,
         timestamp, unix_timestamp,
         likes, replies,
         post_url,
         is_comment_on_comment,
         language_guess,
         post_body) %>% 
  corpus(docid_field = "id",
         text_field = "post_body")

### Tokenization

In [ ]:
tokens_tiktok_comments <- tokens(tiktok_comments_corpus,
                               remove_punct = TRUE,
                               remove_symbols = TRUE,
                               remove_numbers = TRUE,
                               remove_url = TRUE)

### Stopwords

In [ ]:
tokens_tiktok_comments <- tokens_remove(tokens_tiktok_comments,
                                      stopwords("de"))

### DFM

In [ ]:
dfm_tiktok_comments <- dfm(tokens_tiktok_comments)

### Worthäufigkeiten

In [ ]:
dfm_tiktok_comments %>%
  dfm_remove(pattern = c("dass",
                         "@*", "#*")) %>% 
  textstat_frequency(n = 20)

### Top User Mentions

In [ ]:
dfm_users <- dfm_select(dfm_tiktok_comments, pattern = "@*")
topuser <- names(topfeatures(dfm_users, 50))
head(topuser, 10)

### Wortwolke

In [ ]:
dfm_tiktok_comments %>% 
  dfm_remove(pattern = c("dass",
                         "@*", "#*")) %>% 
  dfm_trim(min_termfreq = 50) %>%
  textplot_wordcloud()

### Plot zu Worthäufigkeiten

In [ ]:
tstat_freq <- dfm_tiktok_comments %>% 
  dfm_remove(pattern = c("dass",
                         "@*", "#*")) %>%  
  textstat_frequency(n = 20)

ggplot(tstat_freq, aes(x = frequency, y = reorder(feature, frequency))) +
  geom_col() + 
  labs(x = "Frequency", y = "Feature") +
  scale_x_continuous(expand = expansion(mult = c(0, 0.05)))